In [ ]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
from statsmodels.tsa.arima.model import ARIMA as ARIMA

## Data Preparation

In [ ]:
# Import Data, Train & Holiday_Events
store = pd.read_csv("store-sales-time-series-forecasting/train.csv")
holiday_events = pd.read_excel("store-sales-time-series-forecasting/holidays_events.xlsx")

print(store.head())
print("\n")
print(holiday_events.head())

In [ ]:
# Clean Data, Keep 'date' & 'type'
holiday_events = holiday_events[['date', 'type']]

# Change all 'type' Object to Float 1
holiday_events['type'] = 1

# Change 'type' to 'holiday'
holiday_events = holiday_events.rename(columns={'type': 'holiday'})

# Change Object to DateTime64
store['date'] = pd.to_datetime(store['date'])

# Compile Data
csv = pd.merge(store, holiday_events, on='date', how='outer')
csv = csv.fillna(0)

# Clean Data, Keep any food related 'family'
csv = csv[(csv['family'] == 'BREAD/BAKERY') |
          (csv['family'] == 'DAIRY') |
          (csv['family'] == 'DELI') |
          (csv['family'] == 'EGGS') |
          (csv['family'] == 'FROZEN FOODS') |
          (csv['family'] == 'MEATS') | 
          (csv['family'] == 'POULTRY') |
          (csv['family'] == 'PREPARED FOODS') | 
          (csv['family'] == 'PRODUCE') |
          (csv['family'] == 'SEAFOOD')]

# Clean Data, Keep 'store_nbr' related to Quito
csv = csv[(csv['store_nbr'] == 1.0) |
          (csv['store_nbr'] == 2.0) |
          (csv['store_nbr'] == 3.0) |
          (csv['store_nbr'] == 4.0) |
          (csv['store_nbr'] == 6.0) |
          (csv['store_nbr'] == 7.0) | 
          (csv['store_nbr'] == 8.0) |
          (csv['store_nbr'] == 9.0) | 
          (csv['store_nbr'] == 10.0) |
          (csv['store_nbr'] == 17.0) |
          (csv['store_nbr'] == 18.0) |
          (csv['store_nbr'] == 20.0) |
          (csv['store_nbr'] == 44.0) |
          (csv['store_nbr'] == 45.0) |
          (csv['store_nbr'] == 46.0) |
          (csv['store_nbr'] == 47.0) |
          (csv['store_nbr'] == 48.0) |
          (csv['store_nbr'] == 49.0)]

csv = csv.drop_duplicates()
print(csv.head())

## Exploratory Data Analysis

In [ ]:
# Distribution: Average Sale

# create a 3x3 grid of subplots
f, axes = plt.subplots(3, 3, figsize=(18, 24))

# list of 'family' names for the header
family_names = ['BREAD/BAKERY', 'DAIRY', 'DELI', 'EGGS', 'FROZEN FOODS', 
                'MEATS', 'POULTRY', 'PREPARED FOODS', 'PRODUCE']

# plot histograms for each 'family' in the grid
for i, family in enumerate(family_names):
    row = i // 3
    col = i % 3
    ax = sb.histplot(data=csv[csv['family'] == family], x='sales', ax=axes[row, col], kde=True)
    ax.lines[0].set_color('crimson')
    axes[row, col].set_title(family)
    
f, axes = plt.subplots(1, 1, figsize=(6, 6))
    
# plot histogram for 'SEAFOOD' outside the grid
seafood_plot = sb.histplot(data=csv[csv['family'] == 'SEAFOOD'], x='sales', kde=True)
seafood_plot.lines[0].set_color('crimson')
seafood_plot.set_title('SEAFOOD')

# display the plots
plt.show()

Based on the distribution curves, we can observe that the majority of the curves appear narrow, indicating that the data points are closely clustered together for each year between 2014 and 2017. This suggests that the sales within each category are relatively consistent over time, with little variation from year to year.

Next, we try to plot out the sales of each category over the years.

In [ ]:
# LinePlot (Daily) of 'SEAFOOD'

f, axes = plt.subplots(1, 1, figsize=(6, 6))
seafood_plot = sb.lineplot(data=csv[csv['family'] == 'SEAFOOD'], x='date', y='sales')
seafood_plot.set_title('SEAFOOD')
plt.show()

Due to the high number of dates, it was difficult to analyze the data from the plot. To address this, we resampled the data into weekly intervals instead of daily intervals.

In [ ]:
# convert the index to a datetime index
csv.set_index('date', inplace=True)

In [ ]:
# LinePlot (Weekly)

# create a 3x3 grid of subplots
f, axes = plt.subplots(3, 3, figsize=(18, 24))

# list of family names for the plots
family_names = ['BREAD/BAKERY', 'DAIRY', 'DELI', 'EGGS', 'FROZEN FOODS', 
                'MEATS', 'POULTRY', 'PREPARED FOODS', 'PRODUCE']

# iterate through the families and plot the weekly sales for each
for i, family in enumerate(family_names):
    row = i // 3
    col = i % 3
    # filter for the family
    family_data = csv[csv['family'] == family]
    # group by week and calculate mean sales
    weekly_sales = family_data.resample('W').agg({'sales':'mean'})
    # plot the line chart
    sb.lineplot(data=weekly_sales, x=weekly_sales.index, y='sales', ax=axes[row, col])
    # set the title for the subplot
    axes[row, col].set_title(family)
    # set the x-label for the subplot
    axes[row, col].set_xlabel('Week')
    # set the y-label for the subplot
    axes[row, col].set_ylabel('Sales')

# filter for the 'SEAFOOD' family
family_data = csv[csv['family'] == 'SEAFOOD']

# group the data by week and calculate the mean sales for each week
weekly_sales = family_data.resample('W').agg({'sales':'mean'})

# create a line plot of weekly sales for the 'SEAFOOD' family
f, ax = plt.subplots(1, 1, figsize=(12, 6))
sb.lineplot(data=weekly_sales, x=weekly_sales.index, y='sales', ax=ax)
ax.set_title('SEAFOOD')
ax.set_xlabel('Week')
ax.set_ylabel('Sales')
plt.show()

From the time series plots, we can observe a general increase in food sales over the years. This observation supports our earlier findings from the distribution plots.

## Correlation
From the available data, there are other variables that we would like to investigate in order to determine if they have any correlation with food sales. If there is a correlation, we will attempt to incorporate those variables into our prediction model.

In [ ]:
# Correlation of Onpromotion vs Sales (Weekly)

# List of family names for the plots
family_names = ['BREAD/BAKERY', 'DAIRY', 'DELI', 'EGGS', 'FROZEN FOODS', 
                'MEATS', 'POULTRY', 'PREPARED FOODS', 'PRODUCE']

# Create a 3x3 grid of subplots
f, axes = plt.subplots(3, 3, figsize=(18, 24))

# Iterate through the families and plot the weekly sales for each
for i, family in enumerate(family_names):
    row = i // 3
    col = i % 3
    # Filter for the family
    family_data = csv[csv['family'] == family]
    # Group by week and calculate mean sales and onpromotion counts
    weekly_sales = family_data.resample('W').agg({'sales':'mean', 'onpromotion':'sum'})
    # Calculate the correlation coefficient
    corr_coef = weekly_sales['sales'].corr(weekly_sales['onpromotion'])
    # Create the scatter plot with best fit line and correlation coefficient
    sb.regplot(data=weekly_sales, x='sales', y='onpromotion', ax=axes[row, col])
    axes[row, col].set_title(f'{family}: Corr = {corr_coef:.2f}')
    
# Filter for the 'SEAFOOD' family
family_data = csv[csv['family'] == 'SEAFOOD']
# Group by week and calculate mean sales and onpromotion counts
weekly_sales = family_data.resample('W').agg({'sales':'mean', 'onpromotion':'sum'})
# Calculate the correlation coefficient
corr_coef = weekly_sales['sales'].corr(weekly_sales['onpromotion'])
# Create the scatter plot with best fit line and correlation coefficient in title
f, ax = plt.subplots(1, 1, figsize=(12, 6))
sb.regplot(data=weekly_sales, x='sales', y='onpromotion', ax=ax)
ax.set_title(f'SEAFOOD: Corr = {corr_coef:.2f}')
ax.set_xlabel('Sales')
ax.set_ylabel('Onpromotion')

# Show the plot
plt.show()

The plots and correlation calculations suggest that there is a moderate to high correlation for each category. Therefore, it appears that the variable "onpromotion" has an effect on food sales, and we should include it in our predictive model.

In [ ]:
# Boxplot of holiday vs Sales (Weekly) for Store 1

# List of family names for the plots
family_names = ['BREAD/BAKERY', 'DAIRY', 'DELI', 'EGGS', 'FROZEN FOODS', 
                'MEATS', 'POULTRY', 'PREPARED FOODS', 'PRODUCE']

# Create a 3x3 grid of subplots
f, axes = plt.subplots(3, 3, figsize=(18, 24))

# Iterate through the families and plot the weekly sales for each
for i, family in enumerate(family_names):
    row = i // 3
    col = i % 3
    # Filter for the family
    family_data = csv[csv['family'] == family]
    family_data = family_data.loc[family_data['store_nbr'] == 1.0]
    # Group by week and calculate mean sales and onpromotion counts
    weekly_sales = family_data.resample('W').agg({'sales':'mean', 'holiday':'sum'})
    # Calculate the correlation coefficient
    corr_coef = weekly_sales['sales'].corr(weekly_sales['holiday'])
    # Create the boxplot
    sb.boxplot(data=weekly_sales, x='holiday', y='sales', ax=axes[row, col])
    axes[row, col].set_title(f'{family}: Corr = {corr_coef:.2f}')
    
# Filter for the 'SEAFOOD' family
family_data = csv[csv['family'] == 'SEAFOOD']
family_data = family_data.loc[family_data['store_nbr'] == 1.0]
# Group by week and calculate mean sales and onpromotion counts
weekly_sales = family_data.resample('W').agg({'sales':'mean', 'holiday':'sum'})
# Calculate the correlation coefficient
corr_coef = weekly_sales['sales'].corr(weekly_sales['holiday'])
# Create a box plot to visualize sales distribution
f, ax = plt.subplots(figsize=(10, 6))
sb.boxplot(data=weekly_sales, x='holiday', y='sales', ax=ax)
ax.set_title(f'SEAFOOD: Correlation Coefficient = {corr_coef:.2f}')

# Show the plot
plt.show()

The BoxPlot above suggests that higher holiday occurrences are associated with higher sales, despite the low correlation. Therefore, it is worthwhile to investigate whether this variable should be included in the model.

## Summary of EDA
- In general, it appears that food sales have increased over the years, so we should expect current prices to be slightly higher than before.
- Two variables that we should pay attention to are 'onpromotion', which represents the number of items on promotion for a given week, and 'holiday', which represents the number of holidays occurring in that week. These variables appear to be factors that influence food sales.

# Model

Our strategy for addressing this problem is to train the model to focus exclusively on a single food category. This approach should help the model generalize better than if we were to use data that includes multiple food categories. For the purposes of our example, we have selected the 'BREAD/BAKERY' category.

In [ ]:
# Select one "Bread" category and data from store 1 
df = csv[csv["family"] == "BREAD/BAKERY"]
df = df[df["store_nbr"] == 1]

# Print df info
df.info()
df

In [ ]:
# Plot time series sales data
plt.figure(figsize=(30,30))
plt.title("Sales of Bread products", size=20)
plt.plot(df["sales"])

The plot illustrates the sales of bread/bakery products from January 2013 to September 2014, with time measured in days. Since the time series plot is difficult to interpret with so much data, we will be converting the time axis to weeks and aggregating the daily sales data into weekly means. This will provide a clearer picture of the sales trends over time and aid in data exploration and visualization. However, it's worth noting that the actual training data will still be in daily increments and only the visualization will be scaled down to weekly intervals.

In [ ]:
# Scale date range from days to week. Sales for the week is the mean
resampled_df = df.resample('W').agg({'sales':'mean','onpromotion':'sum','holiday':'sum'})
print(resampled_df)

# Plot time series sales data
plt.figure(figsize=(30,30))
plt.title("Sales of Bread products", size=20)
plt.plot(resampled_df['sales'])

The plot allows us to more easily visualize the trends in the estimated sales data over time. From our observation, it is apparent that the sales price of the time series is non-stationary, which means that the mean and variance of the sales data change as time progresses. This characteristic makes it challenging to forecast future trends using traditional time-series forecasting methods, as time itself becomes a variable that affects the predictions. We will delve further into this issue in subsequent analyses.

In [ ]:
# Data summary
print("MEDIAN: ", df["sales"].median())
print("MEAN: ", df["sales"].mean())
print("SD: ", df["sales"].std())
print("MAX: ", df["sales"].max())
print("Min: ", df["sales"].min())


## Intro to ARIMA model
ARIMA is a popular time series model that combines three different models to analyze time series data: Auto Regressive (AR), Integrated (I), and Moving Average (MA). The name ARIMA reflects this combination of models.

1) Auto Regressive(AR)
It is a regression model that uses past values of the dependent variable (y) to predict future values.

2) Moving Average(MA)
This model analyzes the errors of the predicted values from the past and attempts to make more accurate predictions for the current time period.

3) Integrated (I)
Integrated (I) is not a separate model, but rather a method for dealing with non-stationary data. Stationary data satisfies three conditions: constant mean, constant standard deviation, and no seasonality. 

If the data is non-stationary, the I component is used to transform the data into a stationary state so that the AR and MA models can be applied.


The goal of this notebook is to identify the optimal hyperparameters for the AR, I, and MA components of the ARIMA model. By identifying the best parameters, we can train a model that provides accurate forecasts for our time series data.

# Stationarity of data (I-value)
As previously mentioned, stationarity is a crucial aspect of time series modeling, as it ensures that the model can accurately predict future values. When a time series exhibits a trend, it becomes more challenging to predict future values accurately. Conversely, when the time series is stationary, it is easier to forecast future values since the data does not vary widely.

From the plot above, we can observe that the sales data is not stationary. Specifically, we see that the sales of the store are increasing over time, which suggests the presence of a trend. However, we can confirm the stationarity of the data more accurately by using a statistical test such as the KPSS test.

In [ ]:
from statsmodels.tsa.stattools import kpss
#define KPSS
def kpss_test(timeseries):
    print ('Results of KPSS Test:')
    kpsstest = kpss(timeseries, regression='c')
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
    print (kpss_output)

In [ ]:
# KPSS test
kpss_test(resampled_df["sales"])

The results of the KPSS test can be interpreted as follows:
- If the test statistic is greater than the critical value, we reject the null hypothesis that the series is stationary. 
- If the test statistic is less than the critical value, we fail to reject the null hypothesis and conclude that the series is stationary.

In the case of our data, the test statistic was found to be 1.989824, which exceeded the critical value at all confidence intervals. This result indicates that we must reject the null hypothesis and conclude that the series is not stationary. 

To make the sales data stationary, we will employ a technique called "differencing.

# Differencing

Differencing is a technique used to remove trends from non-stationary time series data. The process involves calculating the differences between consecutive data points by subtracting the value of the previous observation from the value of the next observation.

In [ ]:
# Split train test
size = round(len(resampled_df)*0.8)

train, test = resampled_df[0:size], resampled_df[size:len(resampled_df)]
print(train,'\n', test)
print(len(test))

In [ ]:
difference_df = resampled_df.copy(deep=True)

difference_df["sales"] = difference_df["sales"].diff()
difference_df["sales"] = difference_df["sales"].diff()
difference_df["sales"] = difference_df["sales"].diff()


difference_df["sales"].dropna().plot()
difference_df = difference_df[["sales", "onpromotion"]].dropna()

After performing differencing on the data, we can observe that the trend component of the time series has been removed.

In [ ]:
kpss_test(difference_df["sales"])

After performing differencing on the data, we can re-run the KPSS test to verify that the time series data is now stationary. If the KPSS test statistic is lower than the critical values at all confidence intervals, then we can conclude that the data is now stationary.

The number of times we perform differencing is denoted as the 'I' value in the ARIMA model. In our case, we performed differencing thrice to achieve stationarity in the sales data. It's important to note that while stationarity can be achieved after a single round of differencing, our model showed improved performance with additional rounds of differencing.

# Finding MA value

Now that we have determined the appropriate value for 'I' in our ARIMA model, we will proceed to identify the optimal value for the Moving Average (MA) component. We can use Auto-Correlation Function (ACF) analysis to accomplish this. ACF measures the correlation between a time series and a lagged version of itself. Take for example the image below:


![](ACF.png)

In a time series, the price in a given month (M) can be directly affected by the price in the preceding month (F), which in turn can be directly affected by the price in the month before that (J). This implies that January's price can indirectly impact March's price. However, January's price can also directly impact March's price. The Auto-Correlation Function (ACF) analysis helps us determine the best lagged period to use in our ARIMA model by measuring the correlation between each observation in the time series and its past lags. For instance, we can use the ACF plot to identify whether the price of January (t-2) or the price of February (t-1) better predicts the price of March (t).

(Image taken from: https://towardsdatascience.com/interpreting-acf-and-pacf-plots-for-time-series-forecasting-af0d6db4061c)

In [ ]:
# ACF test
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(difference_df["sales"], lags = 30)

NOTE: When using ACF to identify the optimal value for the MA component, it's important to exclude lag 0 from the analysis since it represents the correlation between the time series and itself. This correlation is always equal to 1 and is therefore not useful in our analysis.

From the ACF plot, we can observe that the highest correlation occurs at lag 1, indicating that the optimal value for the MA component in our ARIMA model is 1. We can ignore any correlation values that fall within the blue box, as these values are not statistically significant.

# Finding AR value
To identify the optimal value for the Auto Regressive (AR) component in our ARIMA model, we can use Partial Auto-Correlation Function (PACF) analysis. PACF measures the correlation between a time series and its own lags, while removing the indirect effects of other lags. (In the example above, how price of Jan affect price of March.)

In [ ]:
# PACF test
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(difference_df["sales"], lags = 30, method = "ols")

As with ACF analysis, when using PACF to identify the optimal value for the AR component, we should exclude lag 0 from the analysis. From our PACF, we observe the highest correlation at lag 1, indicating that the optimal value for the AR component in our ARIMA model is 1. So that is our value for AR.

# Prediction

After determining the optimal values for the AR, I, and MA components of our ARIMA model, we can train the model using the time series data. We can then use the trained model to make predictions about future sales trends.

(Note that the ARIMA function given by statsmodel already does the differencing for us in the function, so we only need to feed in the raw data, not the differenced data.)

In [ ]:
# predict

history = [x for x in train["sales"]]
predictions = []

for t in range(len(test)):
    
    model = ARIMA(history, order=(1,3,1))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(float(yhat))
    obs = test["sales"][t]

    history.append(obs)
    
    print('predicted = %f, expected = %f' % (yhat, obs))

In [ ]:
predictions_series = pd.Series(predictions, index = test.index.shift(-1))
fig,ax = plt.subplots(nrows = 1,ncols = 1,figsize = (15,5))

plt.subplot(1,1,1)
plt.plot(test["sales"],label = 'Expected Values')
plt.plot(predictions_series,label = 'Predicted values');
plt.legend(loc="upper left")
plt.show()

This is the results of the model.

# Evaluate using RMSE

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(predictions_series, test['sales'], squared=False)
print("RMSE: ", rmse)

# Adding of exog variables

So far, we have used past values of the sales data to make predictions about future trends. However, we can also consider other variables that may have an impact on sales, such as the number of items on promotion and the occurrence of holidays. By incorporating these variables into our ARIMA model, we can create a more comprehensive and accurate forecasting tool. This involves adding these variables as exogenous inputs to the ARIMA model.

In [ ]:
# predict

exog_history = [x for x in train["sales"]]
exog_history_exog = [x for x in train[["onpromotion", "holiday"]].to_numpy()]
exog_predictions = []
exog_future_exog = [x for x in test[['onpromotion', "holiday"]].to_numpy()]

for t in range(len(test)):
    
    exog_model = ARIMA(exog_history, order=(1,3,1), exog=exog_history_exog)
    exog_model_fit = exog_model.fit()
    
    exog_output = exog_model_fit.forecast(exog=exog_future_exog[t])
    exog_yhat = exog_output[0]
    exog_predictions.append(float(exog_yhat))
    obs = test["sales"][t]

    exog_history.append(obs)
    exog_history_exog.append(exog_future_exog[t])
    
    print('predicted = %f, expected = %f' % (exog_yhat, obs))

In [ ]:
exog_predictions_series = pd.Series(exog_predictions, index = test.index.shift(-1))
fig,ax = plt.subplots(nrows = 1,ncols = 1,figsize = (15,5))

plt.subplot(1,1,1)
plt.plot(test["sales"],label = 'Expected Values')
plt.plot(exog_predictions_series,label = 'Predicted values');
plt.legend(loc="lower right")
plt.show()

In [ ]:
exog_rmse = mean_squared_error(exog_predictions_series, test['sales'], squared=False)
print("Exog_RMSE: ", exog_rmse)

# Comparison between with Exog variables and without

In [ ]:
plt.subplot(1,1,1)
plt.plot(test["sales"],label = 'Expected Values')
plt.plot(predictions_series,label = 'Predicted values');
plt.plot(exog_predictions_series,label = 'Exog Predicted values');
plt.legend(loc="lower right")
plt.show()

In [ ]:
rmse = mean_squared_error(predictions_series, test['sales'], squared=False)
exog_rmse = mean_squared_error(exog_predictions_series, test['sales'], squared=False)
print("RMSE: ", rmse)
print("Exog_RMSE: ",exog_rmse)

After incorporating the exogenous variables into our ARIMA model, we observed a slight improvement in the model's performance.

# Conclusion

In this notebook, we managed to train a time series model (ARIMA) to predict food sales for specific categories on a weekly basis has the potential to help businesses better plan their inventory and reduce food waste by avoiding over-ordering. This can contribute to achieving Goal 12 of responsible consumption and production.

Through exploratory data analysis (EDA), we were able to identify key trends and variables that impact food sales, such as the number of items on promotion and the occurrence of holidays. By incorporating these variables as exogenous inputs into our ARIMA model, we were able to improve its accuracy and reliability.

Overall, our approach demonstrates the potential benefits of using data-driven insights and machine learning techniques to optimize business operations and improve sales forecasting. By doing so, businesses can make more informed decisions and ultimately contribute to sustainable and responsible consumption and production practices.

---